## Json  parsing and Processing

In [2]:
import json
import os
os.makedirs("data/json_file", exist_ok= True)

In [4]:
# Sample nested JSON data
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

In [6]:
json_data

{'company': 'TechCorp',
 'employees': [{'id': 1,
   'name': 'John Doe',
   'role': 'Software Engineer',
   'skills': ['Python', 'JavaScript', 'React'],
   'projects': [{'name': 'RAG System', 'status': 'In Progress'},
    {'name': 'Data Pipeline', 'status': 'Completed'}]},
  {'id': 2,
   'name': 'Jane Smith',
   'role': 'Data Scientist',
   'skills': ['Python', 'Machine Learning', 'SQL'],
   'projects': [{'name': 'ML Model', 'status': 'In Progress'},
    {'name': 'Analytics Dashboard', 'status': 'Planning'}]}],
 'departments': {'engineering': {'head': 'Mike Johnson',
   'budget': 1000000,
   'team_size': 25},
  'data_science': {'head': 'Sarah Williams',
   'budget': 750000,
   'team_size': 15}}}

In [8]:
with open ("data/json_file/company_data.json", 'w') as f:
    json.dump(json_data, f, indent=2)

In [12]:
# Save JSON Lines format
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open ("data/json_file/event.jsonl", 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item)+ '\n')

## Json Processing Stratergies

In [27]:
from langchain_community.document_loaders import JSONLoader
import json

employee_load = JSONLoader(
    file_path="data/json_file/company_data.json",
    jq_schema= '.employees[]',
    text_content=False
)

em_docs = employee_load.load()
print(f"Load {len(em_docs)} employee docements")
print(f" First employee: {em_docs[0].page_content[:200]}.....")
print(f"\n\n{em_docs}")


Load 2 employee docements
 First employee: {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status".....


[Document(metadata={'source': 'C:\\Users\\oybek\\RAG_udemy\\dataParsing_1\\data\\json_file\\company_data.json', 'seq_num': 1}, page_content='{"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}'), Document(metadata={'source': 'C:\\Users\\oybek\\RAG_udemy\\dataParsing_1\\data\\json_file\\company_data.json', 'seq_num': 2}, page_content='{"id": 2, "name": "Jane Smith", "role": "Data Scientist", "skills": ["Python", "Machine Learning", "SQL"], "projects": [{"name": "ML Model", "status": "In Progress"}, {"name": "Analytics Dashboard", "status": "Planning"}]}')]


In [29]:
# Method 2 

from typing import List
from langchain_core.documents import Document

def process_json(filepath:str) -> List[Document]:
    with open(filepath, "r") as f:
        data = json.load(f)

    documents = []
    # Create  Documents for each employee with full context
    for emp in data.get('employees', []):
        content = f""" Employees Profile:
        Name : {emp["name"]},
        Role : {emp["role"]},
        Skill : {', '.join(emp["skills"])}
        Projects:
        """
        for proj in emp.get("projects", []):
            content += f"\n- {proj['name']} (Status: {proj['status']})"
         
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )
        documents.append(doc)
    return documents
            

In [31]:
process_json("data/json_file/company_data.json")

[Document(metadata={'source': 'data/json_file/company_data.json', 'data_type': 'employee_profile', 'employee_id': 1, 'employee_name': 'John Doe', 'role': 'Software Engineer'}, page_content=' Employees Profile:\n        Name : John Doe,\n        Role : Software Engineer,\n        Skill : Python, JavaScript, React\n        Projects:\n        \n- RAG System (Status: In Progress)\n- Data Pipeline (Status: Completed)'),
 Document(metadata={'source': 'data/json_file/company_data.json', 'data_type': 'employee_profile', 'employee_id': 2, 'employee_name': 'Jane Smith', 'role': 'Data Scientist'}, page_content=' Employees Profile:\n        Name : Jane Smith,\n        Role : Data Scientist,\n        Skill : Python, Machine Learning, SQL\n        Projects:\n        \n- ML Model (Status: In Progress)\n- Analytics Dashboard (Status: Planning)')]